# Part 2: Creating a basic agent

Let's go through the fundamentals of creating an agent.
Here are the key concepts

These are
- **Function Definition** what tools the LLM has available to it.  
- **Function Call** a response from the model indicating a function call should be made.  
- **Function Response** the response we get from the function itself.  

And to tie this all together we need an **LLM Framework**

## How to do this
We're going to implement things one piece at time


## Our goal
Build the LLM framework from scratch so you can see every piece of how the system worls

**Your task is to get a classification of whether this about American football or Australian football**. Ideally we just want the string American or Australian

Start prompting the LLM to see what results you get. Use your intuition to iterate to the right behavior.

In [15]:
import logging

In [18]:
def get_weather(city):
    logging.info("Called weather function %s", city)
    if city.lower() in {"Austin", "Sydney"}:
        return "sunny"
    else:
        return "cloudy"

In [19]:
get_weather("Austin")

'cloudy'

In [64]:
from ollama import chat
from ollama import ChatResponse

model = 'gemma3:4b'

def model_call(prompt):
    
    response: ChatResponse = chat(model=model, messages=[
      {
        'role': 'user',
        'content': prompt,
      },
    ])
    return response['message']['content']

prompt = "Say hello to the class"
single_turn(system_prompt = system_prompt, user_prompt=prompt)

'Hello!\n'

In [65]:
def augmented_model_call(system_prompt, user_prompt):
    combined_prompt = f"{system_prompt}\n{user_prompt}"
    return model_call(combined_prompt)


# This is injected by the LLM application behind the scenes
system_prompt = '''
You have the following functions available
 def get_weather(city: str)
   """Given a city returns the weather for that city""

 If you call this function return the json [{"city": city}]
 otherwise respond normally
'''

# This is the user message
user_prompt = "Say hello to the class"
augmented_model_call(system_prompt, user_prompt)

'Hello!\n'

In [66]:
user_prompt = "What's the weather in Sydney?"
augmented_model_call(system_prompt, user_prompt)

'```json\n{"city": "Sydney"}\n```\n'

In [69]:
user_prompt = "How's the Austin forecast looking today?"
augmented_model_call(system_prompt, user_prompt)

'```json\n{"city": "Austin"}\n```\n'

In [71]:
user_prompt = "How's the London weather looking today?"
augmented_model_call(system_prompt, user_prompt)

'```json\n{"city": "London"}\n```\n'

## Parsing the Response
Now that we can see our model is aware of the tools we need to build a framework around it to process results.

We need a basic condition
1. If there is no function call return the response to the user
2. If there is a function call then we need to
    a. Call the tool to get new information
    b. Either return the response back to the user directly, or reinject it back to hte model

In [76]:
tool_response = '```json\n{"city": "Austin"}\n```\n'

def parse_response(model_response):
    if tool_call := re.search(pattern, tool_response):
        return tool_call.groups(0)

In [77]:
def function_response(city, weather):
    
    return function_response_prompt

In [82]:
def chat_interaction(user_prompt):
    model_response = augmented_model_call(system_prompt, user_prompt)

    #function_call_json = parse_response(model_response)
    function_call_json = {"city":"London"}

    if not function_call_json:
        return model_response


    weather = get_weather(function_call_json["city"])

    # We have a choice here. We oculd return the weather directly to the user
    # But for a nicer experience let's reinject in the LLM to the response
    function_response_prompt = f"The weather in {function_call_json['city']} is {weather}, tell me the weather nicely"

    # We already checked for weather so we don't need to go again
    model_response = model_call(function_response_prompt)
    return model_response

chat_interaction("test")

"Okay, let's see... it's a lovely, cloudy day in London! It’s a bit soft and gentle, perfect for a cozy day. 😊 \n\nWould you like me to tell you anything more about it, like the temperature or if there's a chance of rain?"

## 🎯 Recap: What We Learned  

### Function Calls are nothing magical
* We rely on the models "reasoning" to understand when to make a function call or not

### The model doesn't call a function itself
* It's the framework that handles the function call
